In [ ]:
# examples from https://towardsdatascience.com/understanding-lstm-and-its-quick-implementation-in-keras-for-sentiment-analysis-af410fd85b47

# https://github.com/nsinha280/lstm-on-Yelp-review-data/blob/master/lstm-final.ipynb

In [ ]:
# install tenserflow and test the package
#!pip install tensorflow

#import tensorflow as tf

#a = tf.constant(2)
#b = tf.constant(3)

#c = tf.add(a, b)

#with tf.Session() as sess:
#    result = sess.run(c)
#    print(result)

In [ ]:
!pip install np_utils

In [ ]:
conda install keras

In [ ]:
import numpy as np 
import pandas as pd 
#from tensorflow import keras
import keras
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
import json
import pandas as pd

def convert(x):
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob
json_filename = './tmpData/yelp_dataset/yelp_academic_dataset_review.json'
#json_filename='/Users/nimesh/Downloads/dataset/review.json'
with open(json_filename,'rb') as f:
    data = f.readlines()

df = pd.DataFrame([convert(line) for line in data])

In [ ]:
del(data)

In [ ]:
#data = df[['text', 'stars']]

In [ ]:
del(data)
trialNoAll = 10000
data = df.loc[:trialNoAll][['text', 'stars']]

In [ ]:
print(data)

In [ ]:
#print(data['stars'])
print(data['stars'][0])
print(len(data))

In [ ]:
#tmp_sentiment = []
#for i in range(len(data['stars'])):
#    if data['stars'][i] > 3:
#        tmp_sentiment.append('pos')
#    else:
#        tmp_sentiment.append('neg')
#data = data.assign(sentiment=tmp_sentiment)

In [ ]:
data['sentiment']=['pos' if (x>3) else 'neg' for x in data['stars']]

In [ ]:
data['text']= [x.lower() for x in data['text']]
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [ ]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [ ]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [ ]:
pd.set_option('display.max_colwidth',4)
data[:5]

In [ ]:
data.dtypes

In [ ]:
#data['text']= [x.encode('ascii') for x in data['text']]

In [ ]:
for i in range(len(data['text'])):
    tmp = data['text'][i]
    tmp1 = tmp.encode('ascii')
    data['text'][i] = tmp1
    #print(tmp)
    #print(tmp1)

In [ ]:
#Vectorizing the text
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

In [ ]:
tokenizer.fit_on_texts([x.decode('utf-8') for x in data['text'].values if not isinstance(x, str) ])

In [ ]:
##Vectorizing the text
#tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')
#tokenizer.fit_on_texts(data['text'].values)

In [ ]:
print(type(data['text'].values))

In [ ]:
#print(tokenizer.word_index)
tokenizer

In [ ]:
print(type(data['text']))
print(data['text'].index)
print(data['text'][0])
print(data['text'].values)

In [ ]:

X = tokenizer.texts_to_sequences(data['text'].values.astype(str))
#X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [ ]:
len(data)

In [ ]:
#print(data['text'].values)

  Cloning git://github.com/fchollet/keras.git to c:\users\zhanghqy\appdata\local\temp\pip-req-build-0tj426ug
Note: you may need to restart the kernel to use updated packages.


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [180]:
from keras.layers import Dropout
embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
#model.add(Embedding(2000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.2))
#model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(LSTM(lstm_out))
#model.add(recurrent_dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 924, 128)          256000    
                                                                 
 dropout_6 (Dropout)         (None, 924, 128)          0         
                                                                 
 lstm_7 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [185]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8000, 924) (8000, 2)
(2001, 924) (2001, 2)


In [186]:
print(X_train)

[[   0    0    0 ...   26  430 1356]
 [   0    0    0 ...   25   90  280]
 [   0    0    0 ...   15   31  350]
 ...
 [   0    0    0 ...   21    3  392]
 [   0    0    0 ...    2  162   40]
 [   0    0    0 ...  569   10   38]]


In [188]:
#model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = 1, verbose = 2)
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 1, verbose = 2)




250/250 - 247s - loss: 0.4564 - accuracy: 0.7929 - 247s/epoch - 988ms/step


In [189]:
model

In [190]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

63/63 - 14s - loss: 0.3172 - accuracy: 0.8666 - 14s/epoch - 227ms/step
Score: 0.32
Validation Accuracy: 0.87
